In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
import ray
from ray.train.huggingface.transformers import (
    RayTrainReportCallback,
    prepare_trainer,
)
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer
import datasets
from config import *

In [4]:
dataset = datasets.load_from_disk("/data/lab/project2/squad_v2_tokenized_datasets")
dataset = dataset.train_test_split(test_size=test_size,seed=666)
ray_train_ds = ray.data.from_huggingface(dataset["train"])
ray_eval_ds = ray.data.from_huggingface(dataset["test"])

max_steps_per_epoch = ray_train_ds.count() // (batch_size * num_workers)

In [22]:
def train_func(config):
    model = T5ForConditionalGeneration.from_pretrained(model_dir)
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    
    training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=batch_size,
        overwrite_output_dir=True,
        save_total_limit=save_total_limit,
        logging_dir="./logs",
        num_train_epochs=num_train_epochs,
        report_to="none",
        per_device_eval_batch_size=batch_size,
        save_strategy="steps",
        max_steps=max_steps_per_epoch * 2,
        evaluation_strategy="steps",
        output_dir="./checkpoints",
        warmup_steps=2*eval_steps,
        learning_rate=learning_rate,
        logging_steps=eval_steps,
        predict_with_generate=True,
        gradient_accumulation_steps=gradient_accumulation_steps,
        save_steps=eval_steps,
        eval_steps=eval_steps,
    )
        
    trainer = Seq2SeqTrainer(
        model=model,
        rain_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        args=training_args,
    )
    
    trainer.add_callback(RayTrainReportCallback())
    trainer = prepare_trainer(trainer)
    trainer.train()

scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=False)
ray_trainer = TorchTrainer(
    train_func,
    scaling_config=scaling_config,
    datasets={"train": ray_train_ds, "eval": ray_eval_ds},
)

result = ray_trainer.fit()

2024-05-29 07:12:59,569	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-05-29 07:12:59 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 0/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-29_06-54-58_814481_446177/artifacts/2024-05-29_07-12-59/TorchTrainer_2024-05-29_07-12-59/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(raylet) [2024-05-29 07:13:01,546 E 446902 446902] (raylet) node_manager.cc:3041: 4 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 0ac244d9accc6191d92a062bf4ed6ac2bf10561f4e25aea2208bfce1, IP: 172.17.0.38) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.17.0.38`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


== Status ==
Current time: 2024-05-29 07:13:04 (running for 00:00:05.17)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-29_06-54-58_814481_446177/artifacts/2024-05-29_07-12-59/TorchTrainer_2024-05-29_07-12-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




(TorchTrainer pid=468003) Started distributed worker processes: 
(TorchTrainer pid=468003) - (ip=172.17.0.38, pid=468095) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=468095) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=468095) You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
(RayTrainWorker pid=468095) Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
(RayTrainWorker pid=468095) /root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/training_args.py:147

...

(RayTrainWorker pid=468095) 
 30%|███       | 49/163 [04:42<11:06,  5.85s/it]


== Status ==
Current time: 2024-05-29 07:45:32 (running for 00:32:33.15)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-29_06-54-58_814481_446177/artifacts/2024-05-29_07-12-59/TorchTrainer_2024-05-29_07-12-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




2024-05-29 07:45:33,022	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-05-29 07:45:33,026	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/TorchTrainer_2024-05-29_07-12-59' in 0.0024s.


== Status ==
Current time: 2024-05-29 07:45:33 (running for 00:32:33.45)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-29_06-54-58_814481_446177/artifacts/2024-05-29_07-12-59/TorchTrainer_2024-05-29_07-12-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




(RayTrainWorker pid=468095) 
 31%|███       | 50/163 [04:48<11:00,  5.85s/it]
(RayTrainWorker pid=468095) 
 31%|███▏      | 51/163 [04:54<10:52,  5.82s/it]
2024-05-29 07:45:43,031	INFO tune.py:1039 -- Total run time: 1963.46 seconds (1953.45 seconds for the tuning loop).
2024-05-29 07:45:43,032	WARNING tune.py:1049 -- Training has been interrupted, but the most recent state was saved.
Resume training with: <FrameworkTrainer>.restore(path="/root/ray_results/TorchTrainer_2024-05-29_07-12-59", ...)
